Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [498]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-11 13:48:44--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-11 13:48:44--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K   171KB/s    in 0.4s    

2021-10-11 13:48:45 (171 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [502]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1nQsR62wLqfmw2_kZHjSSisB9R72FByML

Downloading...
From: https://drive.google.com/uc?id=1nQsR62wLqfmw2_kZHjSSisB9R72FByML
To: /content/2018ME10032_A_model.zip
100% 29.1M/29.1M [00:00<00:00, 70.1MB/s]


In [503]:
!unzip 2018ME10032_A_model.zip

Archive:  2018ME10032_A_model.zip
replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: model.txt               
  inflating: vocab.txt               


In [505]:
## Import relevant packages
import os
import re
import numpy as np
import json
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe

In [506]:
torch.manual_seed(0)
np.random.seed(0)

In [507]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [508]:
vocab = torch.load('vocab.txt')

## Creating .CSV Test Data

In [509]:
def csv_creator(path_data='/content/test.data.txt', path_dir='content/', is_label=False, kind='train'):
    ## Reading Data File
    train_list = []
    count = 1
    with open(path_data, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            line = line.split('\t')
            token = line[0]
            entity = line[1]
            pos = line[2].split('-')
            pos1 = pos[0]
            pos2 = pos[1]
            sent1 = line[3]
            ## Punctuation if removed shall change the current order of words 
            ## and thus the significance of the indices given shall be lost
            sent1 = ' '.join(sent1.split())
            sent2 = line[4]
            sent2 = ' '.join(sent2.split())
            train_list.append({"Token": token, "Entity": entity, "Position1": pos1, 
                              "Position2": pos2, "Sentence1": sent1, "Sentence2": sent2})
        f.close()

    ## Creating DataFrame for the data
    train_df = pd.DataFrame(train_list)

    ## Label Encoding
    train_df['Entity'] = np.where(train_df['Entity'] == 'V', 1, 0)

    # Reading Gold Labels
    if is_label:
        label_list = []
        with open(path_gold, 'r') as f:
            label_list = f.read().splitlines()
            f.close()

        ## Label Encoding
        for i in range(len(label_list)):
            if label_list[i] == 'T':
                label_list[i] = 1
            else:
                label_list[i] = 0
        train_df['Label'] = label_list
    
    path_csv = path_dir + str(kind) + '.csv'
    train_df['Index'] = np.arange(0, len(train_df), 1)      ## Adding index to ensure no shuffling
    train_df.to_csv(path_csv, index=True)

In [510]:
csv_creator(path_data='/content/test.data.txt', path_dir = '/content/', is_label=False, kind='test')

## Creating Iteratable Format for Dataset

In [511]:
tokenizer = lambda x: x.split()

In [512]:
text_field = Field(sequential=True, use_vocab=True, tokenize=tokenizer, lower=True, batch_first=True, include_lengths=True)
label_field = Field(sequential=False, use_vocab=False, batch_first=True)

In [513]:
fields = {'Token': ('t', text_field), 'Entity': ('e', label_field), 'Position1': ('p1', label_field), 
          'Position2': ('p2', label_field), 'Sentence1': ('s1', text_field), 'Sentence2': ('s2', text_field), 'Index': ('i', label_field)}

In [514]:
test_data = TabularDataset(
    path = '/content/test.csv',
    format = 'csv',
    fields = fields,
)

In [515]:
text_field.vocab = vocab

In [516]:
len(test_data)

638

In [517]:
print(test_data[0].__dict__.keys())
print(test_data[0].__dict__.values())

dict_keys(['t', 'e', 'p1', 'p2', 's1', 's2', 'i'])
dict_values([['board'], '0', '2', '2', ['room', 'and', 'board', '.'], ['he', 'nailed', 'boards', 'across', 'the', 'windows', '.'], '0'])


In [518]:
test_iterator = BucketIterator(
    test_data,
    batch_size=len(test_data),
    sort_key=lambda x: len(list(x.i)),
    device=device,
    sort=True,
    sort_within_batch=True
)

In [519]:
count = 0
for batch in test_iterator:
    if count < 5:
        print(batch.i)
        count += 1

tensor([637, 636, 635, 634, 633, 632, 631, 630, 629, 628, 627, 626, 625, 624,
        623, 622, 621, 620, 619, 618, 617, 616, 615, 614, 613, 612, 611, 610,
        609, 608, 607, 606, 605, 604, 603, 602, 601, 600, 599, 598, 597, 596,
        595, 594, 593, 592, 591, 590, 589, 588, 587, 586, 585, 584, 583, 582,
        581, 580, 579, 578, 577, 576, 575, 574, 573, 572, 571, 570, 569, 568,
        567, 566, 565, 564, 563, 562, 561, 560, 559, 558, 557, 556, 555, 554,
        553, 552, 551, 550, 549, 548, 547, 546, 545, 544, 543, 542, 541, 540,
        539, 538, 537, 536, 535, 534, 533, 532, 531, 530, 529, 528, 527, 526,
        525, 524, 523, 522, 521, 520, 519, 518, 517, 516, 515, 514, 513, 512,
        511, 510, 509, 508, 507, 506, 505, 504, 503, 502, 501, 500, 499, 498,
        497, 496, 495, 494, 493, 492, 491, 490, 489, 488, 487, 486, 485, 484,
        483, 482, 481, 480, 479, 478, 477, 476, 475, 474, 473, 472, 471, 470,
        469, 468, 467, 466, 465, 464, 463, 462, 461, 460, 459, 4

In [520]:
class BiLSTM_WordMeaningComparison(nn.Module):
    def __init__(self, vocab_size, embed_dim,
                 hidden, n_layers):
        super(BiLSTM_WordMeaningComparison, self).__init__()    ## loading from parent class
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.embed.weight.data.copy_(text_field.vocab.vectors)
        self.embed.weight.requires_grad = False     ## Freezing the embeddings
        self.lstm = nn.LSTM(embed_dim, hidden,
                            num_layers=n_layers,
                            bidirectional=True,     ## makes it a BiLSTM
                            batch_first=True)
        drp1 = 0.5      # dropout probability
        self.dropout1 = nn.Dropout(drp1)
        self.fc1 = nn.Linear(hidden * 2, 32)        ## fully connected layer

    def forward(self, input, actual_batch_len):
        embed_out = self.embed(input)
        hidden = torch.zeros(self.n_layers * 2, input.shape[0], self.hidden, requires_grad=True).to(device)   ## hidden state
        cell = torch.zeros( self.n_layers * 2, input.shape[0], self.hidden, requires_grad=True).to(device)    ## cell state
        out_lstm, (hidden, cell) = self.lstm(embed_out,(hidden, cell)) # Applying BiLSTM Model
        out_lstm = self.dropout1(out_lstm)                             # Applying dropout
        out_lstm = self.fc1(out_lstm)                                  # Applying Fully connected layer
        return out_lstm

## Defining input parameters for the model
vocab_dim = len(text_field.vocab)
embedding_dim = text_field.vocab.vectors.shape[1]
hidden_dim = 256
num_layers = 1
model = BiLSTM_WordMeaningComparison(vocab_dim, embedding_dim, hidden_dim, num_layers)

In [521]:
## Defining Optimizer and Criterion
opt = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CosineEmbeddingLoss()          ## to gain a measure of similarity between the two word embeddings
model.to(device)

BiLSTM_WordMeaningComparison(
  (embed): Embedding(7460, 300)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
)

In [522]:
## Loading saved model
checkpoint = torch.load('model.txt')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()

BiLSTM_WordMeaningComparison(
  (embed): Embedding(7460, 300)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
)

In [523]:
print("model = ", model)
print("optimizer = ", opt)
print("epoch = ", epoch)
print("loss = ", loss)

model =  BiLSTM_WordMeaningComparison(
  (embed): Embedding(7460, 300)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
)
optimizer =  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
epoch =  12
loss =  0.42919013623533575


In [524]:
model.to(device)

BiLSTM_WordMeaningComparison(
  (embed): Embedding(7460, 300)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
)

In [580]:
## Defining Accuracy
def accuracy(preds, y):
    acc = torch.sum(preds == y) / y.shape[0]
    return acc

## Calculating Loss
def calculateLoss(model, batch, criterion):
    ## Running BiLSTM for Sentence 1
    text, text_len = batch.s1
    p1 = batch.p1
    embeddings = model(text, text_len.to(device))

    ## Running BiLSTM for Sentence 2
    text_2, text_len_2 = batch.s2
    p2 = batch.p2
    embeddings_2 = model(text_2, text_len_2.to(device))

    ## Picking out the specific word embeddings from the output for sentence 1
    j = 1
    first = embeddings[0, p1[0], :]
    for i in p1[1:]:
        first = torch.vstack((first, embeddings[j, i, :]))
        j += 1

    ## Picking out the specific word embeddings from the output for sentence 2
    j = 1
    second = embeddings_2[0, p2[0], :]
    for i in p2[1:]:
        second = torch.vstack((second, embeddings_2[j, i, :]))
        j += 1

    ## Generating cosine similarity
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    res = cos(first, second).float()

    for i in range(len(res)):
        if res[i] > 0.645:
          res[i] = 1
        else:
          res[i] = 0
    
    return res

In [581]:
## Generating results
count = 0
with torch.no_grad():
    for batch in test_iterator:
        if count == 0:
            val_results = calculateLoss(model, batch, criterion)
            count += 1
        else:
            val_results = torch.cat((val_results, calculateLoss(model, batch, criterion)), 0)
    print(val_results)

tensor([0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
        1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
        0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        0., 1., 1., 0., 1., 1., 1., 1., 

In [582]:
## Converting results to T and F format
res = []
for i in val_results:
    if i.int() == 0:
        res.append('F')
    else:
        res.append('T')
res.reverse()
print(len(res))
print(res)

638
['T', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 

In [583]:
## Saving results in the given format
with open('output.txt', 'w') as f:
    for item in res:
        f.write("%s\n" % item)

In [584]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-11 13:53:45--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-11 13:53:46--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt.11’

test.gold.txt.11    100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-11 13:53:46 (61.2 MB/s) - ‘test.gold.txt.11’ saved [1276/1276]

Performance =  0.6128526645768025
